In [10]:
%pip install -qqqU langchain pypdf boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

def load_documents():
    document_loader = PyPDFDirectoryLoader("docs")
    return document_loader.load()

len(load_documents())

77

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False
    )

    return text_splitter.split_documents(documents)

In [15]:
from langchain_community.embeddings.bedrock import BedrockEmbeddings

def get_embedding_function():
    embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )
    return embeddings

get_embedding_function()

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x737011303990>, region_name='us-east-1', credentials_profile_name='default', model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)

In [25]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

def get_embedding_function():
    embeddings = OllamaEmbeddings(model="llama3")
    return embeddings

get_embedding_function()

OllamaEmbeddings(base_url='http://localhost:11434', model='llama3', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [26]:
from langchain.vectorstores.chroma import Chroma

def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory="./chroma", embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")


In [27]:
documents = load_documents()
chunks = split_documents(documents)
add_to_chroma(chunks)

Number of existing documents in DB: 0
👉 Adding new documents: 267


In [43]:
from langchain_community.llms.ollama import Ollama
from langchain.prompts import ChatPromptTemplate

query = "Qual os requisitos de turma específica?"

PROMPT_TEMPLATE = """
Responda a questão apenas baseada nesse contexto:

{context}

---

Responda a questão baseada no contexto acima: {question}
"""

db = Chroma(persist_directory="./chroma", embedding_function=get_embedding_function())
results = db.similarity_search_with_score(query, k=5)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

model = Ollama(model="llama3", temperature=.7)
response_text = model.invoke(prompt)

sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)
response_text

Response: Nesse contexto, os requisitos de turma específica são:

* A estudante não pode ter sido suspensa do programa;
* O estudante esteve realizando mobilidade acadêmica em outra instituição e não inclui períodos letivos contados no perfil inicial;
* A carga horária mínima (CHM) e a duração padrão (DP) devem ser atendidas para integralização da estrutura curricular do estudante, de acordo com o Projeto Pedagógico do Curso;
* O estudante regular deve estar vinculado ao curso equivalente ou ao curso correlato na instituição de origem.

Portanto, os requisitos de turma específica são: não suspensão do programa, realização de mobilidade acadêmica em outra instituição, integralização da estrutura curricular e vínculo com o curso equivalente ou correlato.
Sources: ['docs/regulamento-dos-cursos-de-graduacao-da-UFRN-2024.pdf:12:2', 'docs/regulamento-dos-cursos-de-graduacao-da-UFRN-2024.pdf:76:3', 'docs/regulamento-dos-cursos-de-graduacao-da-UFRN-2024.pdf:23:3', 'docs/regulamento-dos-cursos-

'Nesse contexto, os requisitos de turma específica são:\n\n* A estudante não pode ter sido suspensa do programa;\n* O estudante esteve realizando mobilidade acadêmica em outra instituição e não inclui períodos letivos contados no perfil inicial;\n* A carga horária mínima (CHM) e a duração padrão (DP) devem ser atendidas para integralização da estrutura curricular do estudante, de acordo com o Projeto Pedagógico do Curso;\n* O estudante regular deve estar vinculado ao curso equivalente ou ao curso correlato na instituição de origem.\n\nPortanto, os requisitos de turma específica são: não suspensão do programa, realização de mobilidade acadêmica em outra instituição, integralização da estrutura curricular e vínculo com o curso equivalente ou correlato.'